# Checking the affinity of TF to Loci in Regolatory Regions 
In this file, I do the process where I take loci from the MPRA data and check the affinity of the TFs to the ancestral and derived DNA sequences. 

Written by Matanya Wiener

In [ ]:
# First let's import the libraries we will use:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os
import sys

import seaborn as sns
import pandas as pd
from functools import reduce
import utils_matanya as um
import subprocess

In [2]:
# Useful constants
MY_DATA_DIR = "/home/labs/davidgo/matanyaw/data"

MPRA_FILE = "/home/labs/davidgo/Collaboration/humanMPRA/chondrocytes/comparative_analysis_combined/humanMPRA_with_seq_final2.csv"
PBM_FILE = "/home/labs/davidgo/matanyaw/data/pbm_8mer_aggregated_data.csv"
JOBS_OUT_DIR = "/home/labs/davidgo/matanyaw/jobs_outputs"
JOBS_ERR_DIR = "/home/labs/davidgo/matanyaw/jobs_errors"
# TF_2_LOCUS_ZSCORES_DIR = "/home/labs/davidgo/matanyaw/data/tf_2_locus_zscores/"



Loading PBM data

In [3]:
all_8mer_pbm_df = pd.read_csv(PBM_FILE, index_col=0, header=[0,1])
escore_df, median_df, zscore_df = all_8mer_pbm_df['E-score'], all_8mer_pbm_df['Median'], all_8mer_pbm_df['Z-score']
zscore_df

,ARX,Ahctf1_mus_musculus,Alx3_mus_musculus,Alx4_mus_musculus,Ar_mus_musculus,Arid3a_mus_musculus,Arid5a_mus_musculus,Ascl2_mus_musculus,Atf3_mus_musculus,BCL11A,...,TFAP2A_mus_musculus,Tbx2_mus_musculus,Tef_mus_musculus,VAX2,VENTX,VSX1,VSX2,WT1,ZNF200,ZNF655
8-mer,,,,,,,,,,,,,,,,,,,,,
AAAAAAAA,-2.5642,9.0057,2.3401,1.7044,1.1648,2.423686,6.037755,5.100156,2.0726,0.7592,...,0.8049,-0.2041,0.4316,0.4226,-4.0103,1.1738,1.9972,1.3045,1.6922,0.1415
AAAAAAAC,-2.1900,7.2302,1.3416,1.4467,1.1031,-0.021227,3.549164,2.741533,1.5885,-0.9601,...,-0.3715,0.7683,1.4607,0.1454,-2.8133,0.2560,1.8681,0.8530,1.7340,0.0316
AAAAAAAG,-2.9855,6.1877,1.3386,1.0929,-0.0242,1.853928,2.381357,0.994396,2.8981,-0.7073,...,1.5246,0.8301,0.6015,-0.5753,-3.8743,-0.0920,0.8050,1.4324,1.8098,0.5059
AAAAAAAT,-1.8606,8.8501,2.1334,2.3737,0.1240,3.591318,2.863760,2.641339,2.4990,-0.6869,...,1.7388,1.1866,1.4478,1.1230,-1.8313,1.9065,2.6485,1.1602,1.4194,0.9966
AAAAAACA,-2.2716,5.3682,1.6949,1.1549,1.7486,1.395083,3.334386,1.987293,1.7003,-0.2942,...,-1.8648,1.1769,1.4901,0.5406,-1.8409,0.2393,1.8914,1.5113,1.7164,0.2976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCCAAA,-0.2410,2.9411,0.5581,0.2975,-0.0945,-0.002882,1.680976,-0.630805,0.9508,-0.2414,...,1.2079,1.0096,0.2931,1.0831,-0.7304,1.0992,1.2829,0.7699,0.8522,0.6617
TTTCGAAA,-2.3461,1.3296,0.8202,-0.0643,1.5874,0.995079,1.286309,-0.143761,1.7645,-1.5172,...,0.1766,0.7694,1.6737,1.4748,-2.8707,-0.2231,2.3837,0.7933,0.0525,0.4264
TTTGAAAA,-1.9434,3.5111,1.6207,1.2021,1.5820,1.298207,3.294636,0.557449,1.6849,0.0998,...,1.3062,1.7171,1.8915,0.9733,-3.1528,1.5440,2.3192,1.2091,1.5615,0.6381


Loading MPRA Data

In [4]:
full_mpra_df = pd.read_csv(MPRA_FILE)
# We will have a look only at the differencially expressing oligos
mpra_df = full_mpra_df[full_mpra_df['differential_activity'] == True]
mpra_df.reset_index(drop=True, inplace=True)
DIFFERENTIAL_ACTIVE_MPRA_FILE = os.path.join(MY_DATA_DIR, "humanMPRA_with_seq_final2_differential_active.csv")
mpra_df.to_csv(DIFFERENTIAL_ACTIVE_MPRA_FILE, index=False)
mpra_df

,oligo,RNA_DNA_ratio_log2_ancestral,RNA_DNA_ratio_log2_derived,DNA_counts_raw_ancestral,DNA_counts_raw_derived,RNA_counts_raw_ancestral,RNA_counts_raw_derived,barcode_count_ancestral,barcode_count_derived,alpha_activity_estimate_ancestral,...,normalized_activity_estimate_derived,activity_fdr_ancestral,activity_fdr_derived,activity_ancestral,activity_derived,logFC_derived_vs_ancestral,differential_activity_fdr,differential_activity,sequence_ancestral,sequence_derived
0,seq_100038_chr6:4358790-4359059_SCREEN_a3_L1,0.863731,0.991378,441.0,1310.0,2540.0,8233.0,30.0,89.0,4.442910,...,13.975102,1.434529e-26,9.453240e-43,active,active,0.299749,1.584436e-02,True,cagggatgggcacacctggccggctgaggggagcctacagccaggc...,cagggatgggcacacctggccggctgaggggagcctacagccaggc...
1,seq_100065_chr7:138979123-138979392_SCREEN_a3_L1,-0.727351,-0.051537,1894.0,2545.0,3615.0,7760.0,134.0,152.0,1.626701,...,3.538402,7.083968e-01,2.219732e-03,non_active,active,0.316918,4.850811e-06,True,tccattttaccaatcacgaaactgaatcagagagctaagcaacttg...,tccattttaccaatcacgaaactgaatcagagagctaagcaacttg...
2,seq_100070_chr7:79861027-79861296_SCREEN_a3_L1,0.033310,-0.317820,1085.0,549.0,3510.0,1393.0,71.0,54.0,2.536598,...,1.484645,1.606845e-03,2.730638e-01,active,non_active,-0.275615,1.638135e-02,True,ctgtttattgttatactcaagagcacttggtattgttaaagtcgta...,ctgtttattgttatactcaagagcacttggtattgttaaagtcgta...
3,seq_100075_chr16:54376420-54376689_SCREEN_a3_L1,-0.063909,0.348923,1121.0,1485.0,3390.0,5978.0,67.0,94.0,2.370884,...,5.932758,1.232513e-02,3.268023e-08,active,active,0.318055,1.342002e-03,True,aggaaggaggatctctgtgtgtgtgggtcgcccagcctcctgccag...,aggaaggaggatctctgtgtgtgtgggtcgcccagcctcctgccag...
4,seq_100090_chr20:31380149-31380418_SCREEN_a3_L1,0.724860,0.382225,1558.0,2152.0,8139.0,8864.0,97.0,106.0,4.313777,...,6.817569,2.892071e-24,1.244647e-10,active,active,-0.281553,6.067254e-04,True,atgtctgccttttgagtgggaggtgaccactgacattccctagggg...,atgtctgccttttgagtgggaggtgaccactgacattccctagggg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15072,seq_99921_chr11:34262393-34262662_SCREEN_a3_L1,-0.509656,-0.119381,1127.0,784.0,2502.0,2282.0,76.0,69.0,1.829854,...,2.631194,4.778543e-01,3.238074e-02,non_active,active,0.286606,3.722563e-03,True,tggcctcccaaagtgctgggattacaggtgtgagccaccgtacctg...,tggcctcccaaagtgctgggattacaggtgtgagccaccgtacctg...
15073,seq_99930_chr10:128329049-128329318_SCREEN_a3_L1,1.000198,-0.163459,660.0,702.0,4176.0,1982.0,33.0,46.0,4.948771,...,2.531735,1.244198e-36,4.122374e-02,active,active,-0.770237,1.065995e-08,True,ccctgagcttggaaacatagagatgagaaaataagttccctacact...,ccctgagcttggaaacatagagatgagaaaataagttccctacact...
15074,seq_99966_chr21:35967796-35968065_SCREEN_a3_L1,-0.030951,-0.383297,2474.0,1475.0,7652.0,3574.0,182.0,101.0,2.453788,...,1.279978,4.690473e-03,3.446261e-01,active,non_active,-0.230129,2.304378e-03,True,gtgtacagacccatctattcatcatagacttcactgtctgggaatg...,gtgtacagacccatctattcatcatagacttcactgtctgggaatg...
15075,seq_99973_chr14:22846595-22846864_SCREEN_a3_L1,0.667980,-0.637902,359.0,429.0,1806.0,872.0,75.0,81.0,2.867773,...,-0.543097,7.781080e-06,8.414780e-01,active,non_active,-0.305473,1.013274e-02,True,gtcaagttggaacttagattctttttcttgcggaagtgggaggcat...,gtcaagttggaacttagattctttttcttgcggaagtgggaggcat...


In [ ]:
def run_single_job(mpra_csv_file, pbm_csv_file, output_dir, window_size, start_index, end_index,
                   job_name="process_TF_to_loci_files", 
                   jobs_outputs_dir=JOBS_OUT_DIR, 
                   jobs_errors_dir=JOBS_ERR_DIR):
    """
    Run a single job to create the TF to loci files.
    args:
    mpra_csv_file: str, path to the MPRA CSV file
    pbm_csv_file: str, path to the PBM CSV file
    output_dir: str, path to the output directory
    window_size: int, the window size to use (for finding sequences in the PBM file)
    start_index: int, the start index to use (for the MPRA file, which can be split into multiple jobs)
    end_index: int, the end index to use (for the MPRA file, which can be split into multiple jobs)
    job_name: str, the name of the job
    jobs_outputs_dir: str, path to the jobs outputs directory
    jobs_errors_dir: str, path to the jobs errors directory
    """
    
    os.makedirs(jobs_outputs_dir, exist_ok=True)
    os.makedirs(jobs_errors_dir, exist_ok=True)

    TF_2_LOCI_SCRIPT_FILE = "/home/labs/davidgo/matanyaw/backup/create_TF_to_loci_files.py"
    
    command = [
        "bsub",  # Executable
        "-q", "short", 
        "-R", "rusage[mem=2000]",   # Resource requirements in Mb
        "-J", job_name,
        "-o", jobs_outputs_dir,
        "-e", jobs_errors_dir,
        "python", TF_2_LOCI_SCRIPT_FILE,  # The Python script to run
        "--mpra_file", mpra_csv_file,
        "--pbm_file", pbm_csv_file,
        "--output_dir", output_dir,
        "--window_size", str(window_size),
        "--start_index", str(start_index),
        "--end_index", str(end_index),
    ]

    try:
        subprocess.run(command, capture_output=True, text=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running the job: {e.stderr}")
        return None

Test run:

In [6]:
from create_TF_to_loci_files import create_tf_to_loci_files
start_index = 30
end_index = 40
job_name = f"test_partial_csv_read_v2"

# You can Run it as a Job:
# run_single_job(mpra_csv_file=DIFFERENTIAL_ACTIVE_MPRA_FILE,
#         pbm_csv_file=PBM_FILE,
#         output_dir=os.path.join(MY_DATA_DIR, job_name),
#         window_size=8,
#         start_index=start_index,
#         end_index=end_index, 
#         job_name=job_name)

# You can run it as a function:
create_tf_to_loci_files(DIFFERENTIAL_ACTIVE_MPRA_FILE, 
                        PBM_FILE, 
                        os.path.join(MY_DATA_DIR, job_name), 
                        8, 
                        start_index, 
                        end_index)

Done with 31/15077
Done with 32/15077
Done with 33/15077
Done with 34/15077
Done with 35/15077
Done with 36/15077
Done with 37/15077
Done with 38/15077
Done with 39/15077
Done with 40/15077


Here, we send the for all 15K sequences, the 

In [7]:
session_id = 10

Full Run:

In [ ]:
OUTPUT_DIR = os.path.join(MY_DATA_DIR, f"tf_to_locus_zscores_v{session_id}") # ? Change this to the desired output directory
num_jobs = 1500
chunk_size = mpra_df.shape[0] // num_jobs
for i in range(num_jobs):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size
        if i == num_jobs - 1:
            end_index = mpra_df.shape[0]
        job_name = f"job_{i}_sessions_{session_id}"
        run_single_job(mpra_csv_file=DIFFERENTIAL_ACTIVE_MPRA_FILE,
                pbm_csv_file=PBM_FILE,
                output_dir=OUTPUT_DIR,
                window_size=8,
                start_index=start_index,
                end_index=end_index, 
                job_name=job_name,
                jobs_outputs_dir=os.path.join(JOBS_OUT_DIR, f"session_{session_id}"),           # I want each 1500 jobs to have its own output directory
                jobs_errors_dir=os.path.join(JOBS_ERR_DIR, f"session_{session_id}"))
        print(f"Sent job {i}/{num_jobs} for lines: {start_index} to {end_index}, session {session_id}")

print(f"All {num_jobs} Jobs are Sent!")
session_id += 1

Sent job 0/1500 for lines: 0 to 10, session 10
Sent job 1/1500 for lines: 10 to 20, session 10
Sent job 2/1500 for lines: 20 to 30, session 10
Sent job 3/1500 for lines: 30 to 40, session 10
Sent job 4/1500 for lines: 40 to 50, session 10
Sent job 5/1500 for lines: 50 to 60, session 10
Sent job 6/1500 for lines: 60 to 70, session 10
Sent job 7/1500 for lines: 70 to 80, session 10
Sent job 8/1500 for lines: 80 to 90, session 10
Sent job 9/1500 for lines: 90 to 100, session 10
Sent job 10/1500 for lines: 100 to 110, session 10
Sent job 11/1500 for lines: 110 to 120, session 10
Sent job 12/1500 for lines: 120 to 130, session 10
Sent job 13/1500 for lines: 130 to 140, session 10
Sent job 14/1500 for lines: 140 to 150, session 10
Sent job 15/1500 for lines: 150 to 160, session 10
Sent job 16/1500 for lines: 160 to 170, session 10
Sent job 17/1500 for lines: 170 to 180, session 10
Sent job 18/1500 for lines: 180 to 190, session 10
Sent job 19/1500 for lines: 190 to 200, session 10
Sent job 2

## Looking at the over all differences and tendancies
Now we will run over all the loci and see things about the differences in the affinity of TF. 
Here I will create a figure that for every loci will show the biggest differences of the TF binding between Ancestral and Derived variants. 

In [ ]:
# Collecting the relevant files

OVERALL_TF_BINDING_CONCLUSION_DIR = os.path.join(MY_DATA_DIR, "overall_tf_binding_conclusion")
os.makedirs(OVERALL_TF_BINDING_CONCLUSION_DIR, exist_ok=True)

session_id = 10
diff_pbm_file_paths = []

# for root, dirs, files in os.walk(os.path.join(MY_DATA_DIR, f"tf_to_locus_zscores_v10")):
for root, dirs, files in os.walk(os.path.join(MY_DATA_DIR, f"tf_to_locus_zscores_v{session_id}")):
    for file in files:
        if file == "Difference_PBM_filtered.csv":
            diff_pbm_file_paths.append(os.path.join(root, file))

len(diff_pbm_file_paths)

15077

In [15]:
# Creating the DataFrames that contain the most significant TF binding difference per locus
TF_binding_stronger_in_derived = pd.DataFrame()
TF_binding_stronger_in_ancestral = pd.DataFrame()

for file in diff_pbm_file_paths:
    df = pd.read_csv(file, index_col=0)
    locus = file.split("/")[-2]
    max_pos_diff_per_tf = df.max(axis=1)
    max_neg_diff_per_tf = df.min(axis=1)
    max_pos_diff_per_tf.name = locus
    max_neg_diff_per_tf.name = locus
    TF_binding_stronger_in_derived = pd.concat([TF_binding_stronger_in_derived, max_pos_diff_per_tf], axis=1)
    TF_binding_stronger_in_ancestral = pd.concat([TF_binding_stronger_in_ancestral, max_neg_diff_per_tf], axis=1)

TF_binding_stronger_in_derived.to_csv(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_derived_all_loci_v{session_id}.csv"))
TF_binding_stronger_in_ancestral.to_csv(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_ancestral_all_loci_v{session_id}.csv"))

TF_binding_stronger_in_derived

,seq_295193_chr12_54782669-54782938_SCREEN_a2_L3,seq_71981_chr12_67085937-67086206_SCREEN_a2_L1,seq_33133_chr11_119894841-119895110_SCREEN_a1_L1,seq_248781_chr18_75279211-75279480_SCREEN_a3_L2,seq_147743_chr2_135099118-135099387_SCREEN_a1_L2,seq_334414_chr15_67425482-67425751_SCREEN_a3_L3,seq_45763_chr19_35476338-35476607_SCREEN_a1_L1,seq_208942_chr20_32400013-32400282_SCREEN_a2_L2,seq_148431_chr5_149894095-149894364_SCREEN_a1_L2,seq_319422_chr14_74035636-74035905_SCREEN_a2_L3,...,seq_56641_chr18_47836471-47836740_SCREEN_a2_L1,seq_327468_chr2_54281210-54281479_SCREEN_a2_L3,seq_134370_chr17_54676015-54676284_SCREEN_a1_L2,seq_301884_chr1_93428644-93428913_SCREEN_a2_L3,seq_347825_chr12_48147478-48147747_SCREEN_a3_L3,seq_292377_chr2_134103245-134103514_SCREEN_a2_L3,seq_34252_chr7_43331359-43331628_SCREEN_a1_L1,seq_185850_chr21_46469828-46470097_SCREEN_a2_L2,seq_135927_chr12_43281493-43281762_SCREEN_a1_L2,seq_319091_chr18_22724760-22725029_SCREEN_a2_L3
ARX,0.0000,2.7807,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,4.2163,0.7648
Ahctf1_mus_musculus,0.0000,5.3654,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,2.5386,0.0000,0.0,0.0,0.0,0.0,2.5201,6.8531,0.0000
Alx3_mus_musculus,0.0000,2.7213,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,4.5466,0.0000
Alx4_mus_musculus,0.0000,6.9911,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,9.8966,0.0000
Ar_mus_musculus,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,3.5152,0.0,0.0,0.0,0.0,3.8692,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VSX1,0.0000,3.1938,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,5.2351,0.0000
VSX2,0.0000,2.5473,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.6526,0.0000,0.0,0.0,0.0,0.0,0.0000,4.2920,0.0000
WT1,1.5044,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,4.3547,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000
ZNF200,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,...,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000


In [16]:
# Painting the Heatmaps
sns.set(style="whitegrid")

TF_binding_stronger_in_ancestral_for_plot = np.clip(TF_binding_stronger_in_ancestral, -10, 10)
TF_binding_stronger_in_derived_for_plot = np.clip(TF_binding_stronger_in_derived, -10, 10)


# --- Plot & Save Derived Heatmap ---
plt.figure(figsize=(16, 10))
sns.heatmap(
    TF_binding_stronger_in_derived_for_plot,
    cmap="coolwarm",
    center=0,
    # linewidths=0.5,
    # linecolor='gray'
)
plt.title("TF Binding Stronger in Derived", fontsize=14)
plt.xlabel("Locus")
plt.ylabel("Transcription Factor")
plt.tight_layout()
plt.savefig(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_derived_heatmap_clipped_v{session_id}.png"))
plt.close()

# --- Plot & Save Ancestral Heatmap ---
plt.figure(figsize=(16, 10))
sns.heatmap(
    TF_binding_stronger_in_ancestral_for_plot,
    cmap="coolwarm",
    center=0,
    # linewidths=0.5,
    # linecolor='gray'
)
plt.title("TF Binding Stronger in Ancestral", fontsize=14)
plt.xlabel("Locus")
plt.ylabel("Transcription Factor")
plt.tight_layout()
plt.savefig(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_ancestral_heatmap_clipped_v{session_id}.png"))
plt.close()


# Nadav's Idea: Create a Null Hypothesis
Nadav suggested to compare the heatmaps we get with the heatmaps when the loci is active, yet not "Differencially Active" to create a null hypothesys. 
We can see if there is a difference in the binding differences when the suspected enhancers are truely differentially activating the translation and when they don't. 

In [ ]:
# Creating the csv file of the active, yet not differential active loci
active_loci = full_mpra_df[full_mpra_df["differential_activity"].notna()]
active_loci.loc[:, "differential_activity"]
non_differeantial_active_loci = active_loci[active_loci["differential_activity"] == False].sample(n=15000, random_state=42)
NON_DIFFERENTIAL_ACTIVE_MPRA_FILE = os.path.join(MY_DATA_DIR, "humanMPRA_with_seq_final2_non_differential_active.csv")
non_differeantial_active_loci.to_csv(NON_DIFFERENTIAL_ACTIVE_MPRA_FILE, index=False)
non_differeantial_active_loci


,oligo,RNA_DNA_ratio_log2_ancestral,RNA_DNA_ratio_log2_derived,DNA_counts_raw_ancestral,DNA_counts_raw_derived,RNA_counts_raw_ancestral,RNA_counts_raw_derived,barcode_count_ancestral,barcode_count_derived,alpha_activity_estimate_ancestral,...,normalized_activity_estimate_derived,activity_fdr_ancestral,activity_fdr_derived,activity_ancestral,activity_derived,logFC_derived_vs_ancestral,differential_activity_fdr,differential_activity,sequence_ancestral,sequence_derived
126973,seq_222341_chr4:94317488-94317757_SCREEN_a3_L2,0.088088,0.812775,59.0,58.0,175.0,285.0,8.0,6.0,1.750990,...,10.349688,1.554046e-01,1.084302e-23,non_active,active,0.616487,0.111849,False,tttatctgtgggggatgcattccaagacccccagtggatgcctgaa...,tttatctgtgggggatgcattccaagacccccagtggatgcctgaa...
62938,seq_161671_chr8:52765689-52765958_SCREEN_a1_L2,0.355530,-0.261644,240.0,148.0,1041.0,419.0,64.0,31.0,2.171633,...,-0.479539,1.636684e-03,8.305120e-01,active,non_active,-0.172987,0.331967,False,ttcccaaaaaaggacaaagtatttataaacgtaacacacagagatg...,ttcccaaaaaaggacaaagtatttataaacgtaacacacagagatg...
351428,seq_95464_chr2:196763334-196763603_SCREEN_a3_L1,0.074929,-0.457914,57.0,9.0,192.0,22.0,3.0,1.0,2.759462,...,2.007855,5.347407e-05,1.234114e-01,active,non_active,0.260128,0.812192,False,ggctctgagtaaggaatgcagctcatctgacagaagccgaagtgga...,ggctctgagtaaggaatgcagctcatctgacagaagccgaagtgga...
279277,seq_367305_chr19:50312055-50312324_SCREEN_a1_L...,0.621261,0.426472,374.0,309.0,1504.0,1086.0,69.0,65.0,1.739761,...,3.887843,1.430239e-04,6.286328e-04,active,active,-0.075505,0.629746,False,CGCTCCAGAAGCTGCAGGGGGCGTGGTCATTGTGAGGTCGTCATGG...,CGCTCCAGAAGCTGCAGGGGGCGTGGTCATTGTGAGGTCGTCATGG...
172578,seq_265929_chr19:28514118-28514387_SCREEN_a1_L3,-1.039550,-0.022477,8.0,26.0,13.0,84.0,1.0,3.0,1.550593,...,4.193591,8.145705e-01,1.881766e-04,non_active,active,0.594907,0.283096,False,attccactccagttgtaagaaggtttgagcctccctggtatttgtt...,attccactccagttgtaagaaggtttgagcctccctggtatttgtt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237565,seq_327526_chr2:138013598-138013867_SCREEN_a2_L3,0.884326,0.834392,134.0,96.0,826.0,573.0,25.0,16.0,3.225524,...,7.495204,2.784482e-07,1.029193e-12,active,active,-0.031225,0.928860,False,tcttcctgtttttttttcttcttcattactctgtattattttctca...,tcttcctgtttttttttcttcttcattactctgtattattttctca...
192286,seq_284486_chr9:14231828-14232097_SCREEN_a1_L3,-0.393526,0.355441,75.0,98.0,184.0,404.0,16.0,27.0,1.250958,...,4.222847,9.857919e-01,1.668200e-04,non_active,active,0.078156,0.788955,False,ccagttaacaaaaaccctttttgttggagtctaaaacaagaatggg...,ccagttaacaaaaaccctttttgttggagtctaaaacaagaatggg...
160051,seq_254090_chr15:92544809-92545078_SCREEN_a1_L3,0.141182,-0.528117,18.0,45.0,66.0,101.0,9.0,6.0,2.261398,...,-0.339302,1.658086e-02,8.054126e-01,active,non_active,-0.091894,0.841177,False,agctctgggtctcacatgcccccccacccagggagattgtttagtg...,agctctgggtctcacatgcccccccacccagggagattgtttagtg...
105475,seq_201809_chr12:122186470-122186739_SCREEN_a2_L2,0.127060,0.167052,2153.0,2983.0,8100.0,11537.0,432.0,580.0,2.038560,...,2.867822,2.819413e-02,1.748750e-02,active,active,0.057086,0.207819,False,gcaggagagataaggagccttccttacacattcctccatgagccca...,gcaggagagataaggagccttccttacacattcctccatgagccca...


Sending the jobs to create the many files

In [ ]:
OUTPUT_DIR = os.path.join(MY_DATA_DIR, f"non_differential_active_loci_mpra") # ? Change this to the desired output directory
session_id = "non_diff_active"
num_jobs = 1500
chunk_size = mpra_df.shape[0] // num_jobs
for i in range(num_jobs):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size
        if i == num_jobs - 1:
            end_index = mpra_df.shape[0]
        job_name = f"job_{i}_sessions_{session_id}"
        run_single_job(mpra_csv_file=NON_DIFFERENTIAL_ACTIVE_MPRA_FILE,
                pbm_csv_file=PBM_FILE,
                output_dir=OUTPUT_DIR,
                window_size=8,
                start_index=start_index,
                end_index=end_index, 
                job_name=job_name,
                jobs_outputs_dir=os.path.join(JOBS_OUT_DIR, f"session_{session_id}"),           # I want each 1500 jobs to have its own output directory
                jobs_errors_dir=os.path.join(JOBS_ERR_DIR, f"session_{session_id}"))
        print(f"Sent job {i}/{num_jobs} for lines: {start_index} to {end_index}, session {session_id}")

print(f"All {num_jobs} Jobs are Sent!")

Now we will try to get the same overall conclusions

In [ ]:
session_id = "non_diff_active"
diff_pbm_file_paths_non_diff = []

# for root, dirs, files in os.walk(os.path.join(MY_DATA_DIR, f"tf_to_locus_zscores_v10")):
for root, dirs, files in os.walk(os.path.join(MY_DATA_DIR, f"non_differential_active_loci_mpra")):
    for file in files:
        if file == "Difference_PBM_filtered.csv":
            diff_pbm_file_paths_non_diff.append(os.path.join(root, file))

len(diff_pbm_file_paths_non_diff)

15000

In [ ]:
TF_binding_stronger_in_derived_non_diff = pd.DataFrame()
TF_binding_stronger_in_ancestral_non_diff = pd.DataFrame()

for file in diff_pbm_file_paths_non_diff:
    df = pd.read_csv(file, index_col=0)
    locus = file.split("/")[-2]
    max_pos_diff_per_tf = df.max(axis=1)
    max_neg_diff_per_tf = df.min(axis=1)
    max_pos_diff_per_tf.name = locus
    max_neg_diff_per_tf.name = locus
    TF_binding_stronger_in_derived_non_diff = pd.concat([TF_binding_stronger_in_derived_non_diff, max_pos_diff_per_tf], axis=1)
    TF_binding_stronger_in_ancestral_non_diff = pd.concat([TF_binding_stronger_in_ancestral_non_diff, max_neg_diff_per_tf], axis=1)

TF_binding_stronger_in_derived_non_diff.to_csv(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_derived_all_loci_{session_id}.csv"))
TF_binding_stronger_in_ancestral_non_diff.to_csv(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_ancestral_all_loci_{session_id}.csv"))

TF_binding_stronger_in_derived_non_diff

,seq_327284_chr12_102329728-102329997_SCREEN_a2_L3,seq_249187_chr2_173504631-173504900_SCREEN_a3_L2,seq_321615_chr1_76668763-76669032_SCREEN_a2_L3,seq_92763_chr6_159412026-159412295_SCREEN_a3_L1,seq_331339_chr12_45626734-45627003_SCREEN_a3_L3,seq_14111_chr3_48752844-48753113_SCREEN_a1_L1,seq_268907_chr6_109333167-109333436_SCREEN_a1_L3,seq_380491_chr16_54304711-54304980_SCREEN_a1_L4;promoter.orientation.fix;reverse,seq_202971_chr1_15678874-15679143_SCREEN_a2_L2,seq_332475_chr14_35666169-35666438_SCREEN_a3_L3,...,seq_264640_chr12_1189663-1189932_SCREEN_a1_L3,seq_259738_chr1_207261739-207262008_SCREEN_a1_L3,seq_161671_chr8_52765689-52765958_SCREEN_a1_L2,seq_76310_chr2_231545556-231545825_SCREEN_a2_L1,seq_266241_chr2_188257976-188258245_SCREEN_a1_L3,seq_316467_chr13_110317999-110318268_SCREEN_a2_L3,seq_52953_chr8_11272002-11272271_SCREEN_a1_L1,seq_41400_chr13_73823197-73823466_SCREEN_a1_L1,seq_261160_chr14_69931680-69931949_SCREEN_a1_L3,seq_331317_chr10_89907342-89907611_SCREEN_a3_L3
ARX,2.6046,0.0000,0.0,0.9125,0.000,0.3573,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,2.0191,0.0,0.0,0.0,0.0000,0.0000,2.1930
Ahctf1_mus_musculus,1.9234,3.9722,0.0,0.0000,0.000,5.9892,0.0,0.0,0.0,2.306,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,1.3366,1.4561,4.0683
Alx3_mus_musculus,2.5892,0.0000,0.0,0.3245,0.000,1.4047,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000
Alx4_mus_musculus,4.7882,0.0000,0.0,0.0000,0.000,3.4906,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,2.3056,0.0000
Ar_mus_musculus,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.1530,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VSX1,5.1336,0.0000,0.0,1.0010,0.000,0.8216,0.0,0.0,0.0,0.000,...,0.0,0.0181,0.0,4.0512,0.0,0.0,0.0,0.0000,0.5325,0.0000
VSX2,0.0000,0.0000,0.0,0.0000,0.000,0.0000,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.5935
WT1,0.0000,0.0000,0.0,0.0000,2.436,0.0000,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,1.3177
ZNF200,0.0000,0.0000,0.0,4.1630,0.000,0.0000,0.0,0.0,0.0,0.000,...,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000


In [ ]:
sns.set(style="whitegrid")

TF_binding_stronger_in_ancestral_for_plot_non_diff = np.clip(TF_binding_stronger_in_ancestral_non_diff, -10, 10)
TF_binding_stronger_in_derived_for_plot_non_diff = np.clip(TF_binding_stronger_in_derived_non_diff, -10, 10)


# --- Plot & Save Derived Heatmap ---
plt.figure(figsize=(16, 10))
sns.heatmap(
    TF_binding_stronger_in_derived_for_plot_non_diff = np.clip(TF_binding_stronger_in_derived_non_diff, -10, 10)
,
    cmap="coolwarm",
    center=0,
    # linewidths=0.5,
    # linecolor='gray'
)
plt.title("TF Binding Stronger in Derived", fontsize=14)
plt.xlabel("Locus")
plt.ylabel("Transcription Factor")
plt.tight_layout()
plt.savefig(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_derived_heatmap_clipped_{session_id}.png"))
plt.close()

# --- Plot & Save Ancestral Heatmap ---
plt.figure(figsize=(16, 10))
sns.heatmap(
    TF_binding_stronger_in_ancestral_for_plot_non_diff,
    cmap="coolwarm",
    center=0,
    # linewidths=0.5,
    # linecolor='gray'
)
plt.title("TF Binding Stronger in Ancestral", fontsize=14)
plt.xlabel("Locus")
plt.ylabel("Transcription Factor")
plt.tight_layout()
plt.savefig(os.path.join(OVERALL_TF_BINDING_CONCLUSION_DIR, f"TF_binding_stronger_in_ancestral_heatmap_clipped_{session_id}.png"))
plt.close()


The heatmaps of the non differentially active enhancers look quite the same... :,( 